Import fact and dimension tables as dataframes

In [0]:
fact_df = spark.read.format("delta").load("/delta/fact_EV_gold")
dim_df = spark.read.format("delta").load("/delta/dim_demography_gold")

Create views based on the dataframes to enable SQL-based querying

In [0]:
fact_df.createOrReplaceTempView("Fact_EV_Gold")
dim_df.createOrReplaceTempView("Dim_Demography_Gold")

Identify potential locations for new EV charge points

In [0]:
%sql
-- Write CTE to calculate the ratio of electric vehicles to chargers to identify areas with high EV concentration and limited charging infrastructure
;WITH EV_Charger_Ratio AS ( 
  SELECT 
    Town, 
    No_of_licensed_electric_vehicles, 
    No_of_chargers, 
    CASE 
        WHEN No_of_chargers = 0 THEN No_of_licensed_electric_vehicles 
        ELSE ROUND(No_of_licensed_electric_vehicles / No_of_chargers, 1) 
    END AS EV_to_Charger_Ratio 
  FROM Fact_EV_Gold 
) 

-- Join the above CTE with demographic data
SELECT 
  A.Town, 
  B.Population_density AS Population_density_in_persons_per_sq_km, 
  A.No_of_licensed_electric_vehicles, 
  A.No_of_chargers, 
  A.EV_to_Charger_Ratio 
FROM 
  EV_Charger_Ratio A 
INNER JOIN 
  Dim_Demography_Gold B 
ON 
  A.Town = B.Town 
ORDER BY 
  A.EV_to_Charger_Ratio DESC, 
  B.Population_density DESC;

Town,Population_density_in_persons_per_sq_km,No_of_licensed_electric_vehicles,No_of_chargers,EV_to_Charger_Ratio
Tallaght,50889.8,55927,11,5084.3
Lucan,8362.6,37048,8,4631.0
Clondalkin,17512.8,26391,43,613.7
Rathfarnham,21548.9,1130,13,86.9
Firhouse,11888.6,2,4,0.5


Databricks visualization. Run in Databricks to view.